# Sample Size Calculator (WIP)
Purpose: we want a version of the [Periscope dashboard](https://app.periscopedata.com/app/adrise:tubi/676521/(Official)-Experimentation-Sample-Size-Calculator), but with additional flexibility of filtering for a specific set of users. 

The very high level idea of this notebook is:
1. Dynamically generate a SQL query based on a set of user-generated inputs. Run the query on Redshift to pull into a dataframe.
2. Run through the sample size calculations (with the ability for the user to set parameters). Output a table that displays sample required for all chosen platforms. 

The bulk of the work is focused on adding flexibility to #1

In [1]:
import tubi_data_runtime as tdr
# import sample_size_calc_utils as sc

import math
import pandas as pd
import numpy as np
from datetime import date
from statsmodels.stats.power import tt_ind_solve_power

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# User filtering
The goal of this section is to pull a list of specific users that are eligible for the experiment.
- We want this to be dynamic based on the complexity of any filters (see 3 levels below)
- We also want this to be interactable, to minimize the amount of adhoc SQL coding

#### 3 different levels, in order (lowest to highest) of complexity:
1. device_metric_daily
    - all_metric_hourly covers the same metrics/attributes available for filtering, although device_metric_daily will be more performant
2. all_metric_hourly 
3. analytics_richevent
    - using events level data adds infinite flexibilty, but makes this problem much harder
    
For our first prototype, let's just use all_metric_hourly to keep it simple

In [9]:
class filter_generator(object):
    
    def attribute_conditions_choices(self):
        attribute_conditions = [
            '=',
            '<>',
            'IN',
            'IS',
            'IS NOT'
        ]
        return attribute_conditions 
    
    def metric_conditions_choices(self):
        metric_conditions =  [
            '>',
            '<',
            '>=',
            '<=',
            'BETWEEN'
        ] + self.attribute_conditions_choices()
        return metric_conditions 

    def filter_attributes_choices(self):
        # Possible choices for attribute filtering (from all_metric_hourly): 
        filter_attributes = [
                'no filters',
                'user_id',
                'device_id',
                'platform',
                'platform_type',
                'country',
                'region',
                'city',
                'dma',
                'os',
                'os_version',
                'manufacturer',
                'app_mode',
                'app_version',
                'device_language',
                'content_id',
                'program_id',
                'content_type',
                'tvt_sec' # note: here tvt_sec is treated as an attribute rather than a cumulative metric
            ]
        return filter_attributes 

    def filter_metrics_choices(self):
    # Possible choices for metric filtering (from all_metric_hourly): 
        filter_metrics = [
            'no filters',
            'tvt_sec',
            'movie_non_autoplay_tvt_sec',
            'series_non_autoplay_tvt_sec',
            'autoplay_tvt_sec',
            'non_autoplay_tvt_sec',
            'series_tvt_sec',
            'movie_tvt_sec',
            'series_autoplay_tvt_sec',
            'movie_autoplay_tvt_sec',
            'visit_total_count',
            'view_total_count',
            'autoplay_view_total_count',
            'non_autoplay_view_total_count',
            'series_view_total_count',
            'movie_view_total_count',
            'autoplay_movie_view_total_count',
            'autoplay_series_view_total_count',
            'complete_5p_total_count',
            'complete_30p_total_count',
            'complete_70p_total_count',
            'complete_90p_total_count',
            'episode_complete_30p_total_count',
            'episode_complete_70p_total_count',
            'episode_complete_90p_total_count',
            'movie_complete_30p_total_count',
            'movie_complete_70p_total_count',
            'movie_complete_90p_total_count',
            'ad_impression_total_count',
            'ad_break_total_count',
            'seek_total_count',
            'pause_total_count',
            'subtitles_total_count',
            'search_total_count',
            'user_signup_count',
            'device_registration_count',
            'signup_or_registration_activity_count',
            'cast_count',
            'add_to_queue_total_count',
            'details_page_visit_total_count',
            'onboarding_page_visit_total_count',
            'home_page_visit_total_count',
            'browse_page_visit_total_count',
            'category_page_visit_total_count',
            'trailer_start_count',
            'unattributed_tvt_sec',
            'linear_tvt_sec',
            'linear_view_total_count'
        ]
        return filter_metrics 
    
    def base_amh_query(self):
        amh_filter_query = """
        WITH elig_devices as (
            -- Pull list of devices that were active (has any row; don't need TVT >0) in the past 2 weeks
            -- Using all_metric_hourly for additional filters
            SELECT DISTINCT device_id
            FROM tubidw.all_metric_hourly
            WHERE DATE_TRUNC('week',hs) >= dateadd('week',-2,DATE_TRUNC('week',GETDATE()))
            AND DATE_TRUNC('week',hs) < DATE_TRUNC('week',GETDATE())
            {attr_filter} -- attribute filters dynamically populate here

        --     for example:
        --     AND user_id is not null AND device_id <> user_id   -- Guest vs signed in device
        --     AND platform IN ('ROKU', 'AMAZON')                 -- Platform/Platform type specific
        --     AND country in ('US')                              -- Geo specific
        --     AND os IN ('abcdefg')                              -- OS/version specific
        --     AND content_id IN () AND tvt_sec > 0               -- Browsed/watched specific content/content type
        --     TODO: currently can't get a metric/attribute combo filter, like "devices that watched at least 50% of a specific content_id"
        )

        -- The next 3 CTEs are a waste of processing if cumul_filter_metric is not used.
        -- TODO: Figure out some way to make this dynamic, based on if cumul_filter_metric is used or not
        , elig_device_metrics as (
            -- For eligible devices, pull their whole history
            SELECT DISTINCT
                d.device_id,
                d.device_first_seen_ts,
                d.device_first_view_ts,
                d.platform,
                d.platform_type,
                d.ds,

                -- For filtering devices
                {cumul_filter_metric} as daily_filter_metric,

                -- For calculating metrics
                d.tvt_sec,
                d.signup_or_registration_activity_count,
                d.visit_total_count
            FROM tubidw.device_metric_daily as d
            JOIN elig_devices as e
                ON d.device_id = e.device_id
        )

        , elig_device_cumul_filter as (
            SELECT *, sum(daily_filter_metric) OVER (PARTITION BY device_id, platform_type, platform ORDER BY ds rows between unbounded preceding and current row) as cumul_filter_metric
            FROM elig_device_metrics
        )

        , elig_devices2 as (
            SELECT device_id
            FROM elig_device_cumul_filter

            -- cumulative metric filters dynamically populate below 
            WHERE 1=1                                  
            {metric_filter_where}                      -- for > (greater than) filters, we can use "where"
            -- AND cumul_filter_metric >= 3600.0       -- example: at least 60 mins of cumulative TVT

            GROUP BY 1                                 -- using a group by instead of distinct opens up filtering with "having"

            HAVING 1=1                                 
            {metric_filter_having}                     -- for < (less than) filters, need to use a "having" filter with an aggregation on the metric
            -- AND max(cumul_filter_metric) <= 3600.0  -- example: less than 60 mins of cumulative TVT
        )
        """
        return amh_filter_query
    
    
    def make_sql_where_string(self, field, condition, value):
        if field == 'no filters': 
            return ''
        else: 
            if (condition == '<') | (condition == '<=') | (condition == 'BETWEEN'):
            # for < (less than) filters, need to use a "having" filter with an aggregation on the metric
            # for now, the only aggregation is "MAX" but might want to open up to others in the future
                return 'AND ' + 'MAX(' + field + ')' + ' ' + condition + ' ' + value + ''
            else:
                return 'AND ' + field + ' ' + condition + ' ' + value + ''
    
    
    def set_metric_filter_sql_inputs(self, metric_sql):
        # initialize sql strings
        cumul_metric_str = metric_sql_having = metric_sql_where = ''

        if (metric_sql.children[1].value == '<') | (metric_sql.children[1].value == '<=') | (metric_sql.children[1].value == 'BETWEEN'):
            cumul_metric_str = metric_sql.children[0].value
            metric_sql_having = metric_sql.result
        else:
            cumul_metric_str = 0
            metric_sql_where = metric_sql.result
        
        return [metric_sql_where, cumul_metric_str, metric_sql_having]

    
    def generate_filter_cte(self, attribute_sql, metric_sql):
#         base_query = getattr(self, 'base_amh_query')()
#         base_query_inputs = getattr(self, 'set_metric_filter_sql_inputs')(metric_sql)
        base_query = self.base_amh_query()
        base_query_inputs = self.set_metric_filter_sql_inputs(metric_sql)
        
        elig_devices2 = base_query.format(attr_filter = attribute_sql.result,
                                          metric_filter_where = base_query_inputs[0],
                                          cumul_filter_metric = base_query_inputs[1],
                                          metric_filter_having = base_query_inputs[2])
        
        
        return elig_devices2

In [3]:
# TODO: add in capability to do events level filtering 

# Raw user data CTE
Catch-all CTE to pull a list of standard metrics of active devices in the last 4 weeks, from device_metric_daily. 
- In the future, we may want to improve this to allow flexibility for more complex metrics not available in device_metric_daily 
- ie. verification rates can only be calculated from analytics_richevent using is_confirmed = 't'

In [4]:
class raw_user_data(object):

    def generate_raw_user_data_cte(self):
        return """
            , raw_user_data AS (
              SELECT DISTINCT
                     a.device_id,
                     device_first_seen_ts,
                     ds,
                     platform_type,
                     platform,
                     GETDATE() AS last_exposure_ds,
                     DATEADD('week', -2, DATE_TRUNC('week', last_exposure_ds)) AS first_exposure_ds,
                     -- Metrics
                     tvt_sec,
                     signup_or_registration_activity_count,
                     visit_total_count
              FROM elig_device_metrics as a
              JOIN elig_devices2 as e    -- TODO: make this dynamic, based on if cumul_filter_metric is used or not
                ON a.device_id = e.device_id  
              WHERE DATE_TRUNC('week',ds) >= dateadd('week', -4, DATE_TRUNC('week',GETDATE()))
                AND DATE_TRUNC('week',ds) < DATE_TRUNC('week', GETDATE())
            )
        """

# User data CTE
Dynamic CTE that calculates a specific user-chosen metric.

In [5]:
class metric_switcher(object):
    
    def generate_user_data_cte(self, metric): # <-- whatever you input here in "metric" will choose one of the CTEs below      
        # Get the method from 'self'. Default to a lambda.
        method = getattr(self, metric, lambda: "Invalid metric")
        # Call the method as we return it
        return method()
    
    def possible_metrics(self):
        # Possible metrics to use for MDE (same as current calculator)
        # may want to make this consistent with the primary metrics available in exp dash in the future
        metrics = [
            'all_tvt_hours',
            'capped_tvt',
            'new_viewer_first_day_capped_tvt',
            'registrations',
            'visits',
            'viewer_conversion',
            'new_viewer_first_day_conversion',
            'new_user_1_to_8_days_retained',
            'all_user_retained_in_experiment_timeframe',
            'ad_impressions'
        ]
        return metrics
        
    def choose_metric(self, metric):
        return metric

    def all_tvt_hours(self):
        return """
        , user_data AS (
          SELECT 
            device_id, ds, platform_type, platform, device_first_seen_ts, first_exposure_ds, 
            'all_tvt_hours'::text AS metric_name,
            'SUM'::text AS metric_collection_method, 
            tvt_sec / 3600.0 AS metric_value
          FROM raw_user_data
        )
        """

    def capped_tvt(self):
        return """
        , user_data AS (
          SELECT 
            device_id, ds, platform_type, platform, device_first_seen_ts, first_exposure_ds, 
            'capped_tvt'::text AS metric_name,
            'SUM'::text AS metric_collection_method, 
            LEAST(tvt_sec / 3600.0, 4.0) AS metric_value 
          FROM raw_user_data
        )
        """

    def new_viewer_first_day_capped_tvt(self):
        return """
        , user_data AS (
          SELECT 
            device_id, ds, platform_type, platform, device_first_seen_ts, first_exposure_ds, 
            'new_viewer_first_day_capped_tvt'::text AS metric_name,
            'SUM'::text AS metric_collection_method, 
            LEAST(tvt_sec / 3600.0, 4.0) AS metric_value 
          FROM raw_user_data
          WHERE ds >= DATE_TRUNC('day', device_first_seen_ts) AND ds < device_first_seen_ts + INTERVAL '1 day'
        )
        """

    def registrations(self):
        return """
        , user_data AS (
          SELECT 
            device_id, ds, platform_type, platform, device_first_seen_ts, first_exposure_ds, 
            'registrations'::text AS metric_name,
            'SUM'::text AS metric_collection_method, 
            signup_or_registration_activity_count AS metric_value
          FROM raw_user_data
        )
        """

    def visits(self):
        return """
        , user_data AS (
          SELECT 
            device_id, ds, platform_type, platform, device_first_seen_ts, first_exposure_ds, 
            'visits'::text AS metric_name,
            'SUM'::text AS metric_collection_method, 
            visit_total_count AS metric_value 
          FROM raw_user_data
        )
        """

    def viewer_conversion(self):
        return """
        , user_data AS (
          SELECT 
            device_id, ds, platform_type, platform, device_first_seen_ts, first_exposure_ds, 
            'viewer_conversion'::text AS metric_name,
            'MAX'::text AS metric_collection_method, 
            CASE WHEN tvt_sec > 10 THEN 1.0 ELSE 0.0 END AS metric_value 
          FROM raw_user_data
        )
        """

    def new_viewer_first_day_conversion(self):
        return """
        , user_data AS (
          SELECT 
            device_id, ds, platform_type, platform, device_first_seen_ts, first_exposure_ds, 
            'new_viewer_first_day_conversion'::text AS metric_name,
            'MAX'::text AS metric_collection_method, 
            CASE WHEN tvt_sec > 10 THEN 1.0 ELSE 0.0 END AS metric_value
          FROM raw_user_data
          WHERE ds >= DATE_TRUNC('day', device_first_seen_ts) AND ds < device_first_seen_ts + INTERVAL '1 day'
        )
        """

    def new_user_1_to_8_days_retained(self):
        return """
        , user_data AS (
          SELECT 
            device_id, ds, platform_type, platform, device_first_seen_ts, first_exposure_ds, 
            'new_user_1_to_8_days_retained'::text AS metric_name,
            'MAX'::text AS metric_collection_method, 
            CASE WHEN ds > device_first_seen_ts + INTERVAL '1 day' AND tvt_sec > 10 THEN 1.0 ELSE 0.0 END AS metric_value 
          FROM raw_user_data
          WHERE ds >= DATE_TRUNC('day', device_first_seen_ts) AND ds < device_first_seen_ts + INTERVAL '8 day'
        )
        """

    def all_user_retained_in_experiment_timeframe(self):
        return """
        , user_data AS (
          SELECT DISTINCT 
            device_id, ds, platform_type, platform, device_first_seen_ts, first_exposure_ds, 
            'all_user_retained_in_experiment_timeframe'::text AS metric_name,
            'SUMGREATERTHAN'::text AS metric_collection_method,
            1.0 AS metric_value
          FROM raw_user_data
          WHERE tvt_sec > 10
        )
        """

    def ad_impressions(self):
        return """
        , ad_impressions_data AS (
            SELECT ds,
                   device_id,
                   device_first_seen_ts,
                   {{ platform_type('platform') }} AS platform_type,
                   platform,
                   GETDATE() AS last_exposure_ds,
                   DATEADD('week', -2, DATE_TRUNC('week', last_exposure_ds)) AS first_exposure_ds,
                   COALESCE(ad_impression_total_count, 0)::float AS ad_impression_total_count,
                   COALESCE(gross_revenue, 0)::float AS gross_revenue
            FROM tubidw.revenue_bydevice_daily
            WHERE DATE_TRUNC('week',ds) >= dateadd('week', -4, DATE_TRUNC('week',GETDATE()))
              AND DATE_TRUNC('week',ds) < DATE_TRUNC('week', GETDATE())
        )

        , device_data_impressions AS (
            SELECT d.device_id,
                   d.ds,
                   d.platform_type,
                   d.platform,
                   d.device_first_seen_ts,
                   d.first_exposure_ds,
                   SUM(COALESCE(rev.ad_impression_total_count, 0))::float AS ad_impression_total_count,
                   SUM(COALESCE(rev.gross_revenue, 0))::float AS gross_revenue
            FROM ad_impressions_data AS rev
              RIGHT JOIN raw_user_data AS d
                ON d.device_id = rev.device_id
                AND d.ds = rev.ds
            GROUP BY 1, 2, 3, 4, 5, 6
        )

          -- Impressions
        , user_data AS (
          SELECT 
            device_id, ds, platform_type, platform, device_first_seen_ts, first_exposure_ds, 
            'ad_impressions' AS metric_name,
            'SUM' AS metric_collection_method,
            ad_impression_total_count AS metric_value
          FROM device_data_impressions
        )
        """

# Metric Summary CTE
Catch-all code that allows us to summarize/prep the data for CUPED

In [6]:
class metric_summary_cte(object):

    def generate_metric_summary_cte(self):
        return """
            , metrics AS (
              SELECT DISTINCT
                     user_data.device_id,
                     platform_type,
                     platform,
                     metric_name,
                     CASE
                       WHEN metric_collection_method = 'SUM' THEN SUM(CASE WHEN user_data.ds >= user_data.first_exposure_ds THEN metric_value ELSE 0 END) OVER
                        (PARTITION BY user_data.device_id, metric_name)
                       WHEN metric_collection_method = 'MAX' THEN MAX(CASE WHEN user_data.ds >= user_data.first_exposure_ds THEN metric_value ELSE 0 END) OVER
                        (PARTITION BY user_data.device_id, metric_name)
                      WHEN metric_collection_method = 'AVG' THEN AVG(CASE WHEN user_data.ds >= user_data.first_exposure_ds THEN metric_value ELSE NULL END) OVER
                        (PARTITION BY user_data.device_id, metric_name)
                      WHEN metric_collection_method = 'SUMGREATERTHAN' THEN CASE WHEN (SUM(CASE WHEN user_data.ds >= user_data.first_exposure_ds THEN metric_value ELSE 0 END) OVER
                        (PARTITION BY user_data.device_id, metric_name)) > 1 THEN 1.0 ELSE 0.0 END
                      ELSE 0 END
                     AS metric_result,
                    CASE
                          WHEN metric_collection_method = 'SUM' THEN
                              SUM(CASE WHEN user_data.ds < user_data.first_exposure_ds THEN metric_value ELSE
                          (CASE WHEN device_first_seen_ts < user_data.first_exposure_ds - interval '14 day' THEN 0 ELSE NULL END) END) OVER
                          (PARTITION BY user_data.device_id, metric_name)
                          WHEN metric_collection_method = 'MAX' THEN
                              MAX(CASE WHEN user_data.ds < user_data.first_exposure_ds THEN metric_value ELSE
                          (CASE WHEN device_first_seen_ts < user_data.first_exposure_ds - interval '14 day' THEN 0 ELSE NULL END) END) OVER
                          (PARTITION BY user_data.device_id, metric_name)
                          WHEN metric_collection_method = 'AVG' THEN
                              AVG(CASE WHEN user_data.ds < user_data.first_exposure_ds THEN metric_value ELSE
                          (CASE WHEN device_first_seen_ts < user_data.first_exposure_ds - interval '14 day' THEN 0 ELSE NULL END) END) OVER
                          (PARTITION BY user_data.device_id, metric_name)
                    WHEN metric_collection_method = 'SUMGREATERTHAN' THEN
                      CASE WHEN (SUM(CASE WHEN user_data.ds < user_data.first_exposure_ds THEN metric_value ELSE
                          (CASE WHEN device_first_seen_ts < user_data.first_exposure_ds - interval '14 day' THEN 0 ELSE NULL END) END) OVER
                          (PARTITION BY user_data.device_id, metric_name)
                          ) > 1 THEN 1 ELSE 0 END
                          ELSE 0 END AS metric_covariate
              FROM user_data
            )
        """

# CUPED CTEs
Catch-all CTE to calculate CUPED for all platforms, platform types, and all Tubi

In [11]:
class cuped_cte(object):

    def generate_cuped_cte(self):
        return """
            -- Cuped values
            , cuped_values_1 AS (
              SELECT
                *,
                AVG(metric_covariate) OVER (PARTITION BY metric_name, platform_type) AS before_covariate_average,
                AVG(metric_result) OVER (PARTITION BY metric_name, platform_type) AS after_covariate_average,
                STDDEV(metric_covariate) OVER (PARTITION BY metric_name, platform_type) AS covariate_standard_dev,
                AVG(metric_covariate) OVER (PARTITION BY metric_name) AS before_covariate_average_total,
                AVG(metric_result) OVER (PARTITION BY metric_name) AS after_covariate_average_total,
                STDDEV(metric_result) OVER (PARTITION BY metric_name) AS covariate_standard_dev_total
              FROM metrics
            )

            , cuped_values_2 AS (
              SELECT
                *,
                AVG(metric_covariate) OVER (PARTITION BY metric_name, platform) AS before_covariate_average,
                AVG(metric_result) OVER (PARTITION BY metric_name, platform) AS after_covariate_average,
                STDDEV(metric_covariate) OVER (PARTITION BY metric_name, platform) AS covariate_standard_dev
              FROM metrics
              WHERE platform in ('ROKU','AMAZON','IPHONE','IPAD','ANDROID','SONY','PS4','COMCAST','VIZIO','XBOXONE','SAMSUNG','COX')
            )

            , cuped_data_1 AS (
              SELECT
                metric_name,
                AVG(metric_covariate) AS covariate_mean,
                1.0 * SUM((metric_covariate - before_covariate_average_total)*(metric_result - after_covariate_average_total)) / NULLIF(STDDEV(metric_covariate)*STDDEV(metric_covariate) * COUNT(*), 0) AS theta
              FROM cuped_values_1
              GROUP BY 1
            )

            , cuped_data_2 as (
              SELECT
                metric_name,
                platform_type,
                AVG(metric_covariate) AS covariate_mean,
                1.0 * SUM((metric_covariate - before_covariate_average)*(metric_result - after_covariate_average)) / NULLIF(STDDEV(metric_covariate)*STDDEV(metric_covariate) * COUNT(*), 0) AS theta
              FROM cuped_values_1
              WHERE platform_type NOT IN ('WEB')  -- to avoid duplicates with platform
              GROUP BY 1, 2
            )

            , cuped_data_3 as (
              SELECT
                metric_name,
                platform,
                AVG(metric_covariate) AS covariate_mean,
                1.0 * SUM((metric_covariate - before_covariate_average)*(metric_result - after_covariate_average)) / NULLIF(STDDEV(metric_covariate)*STDDEV(metric_covariate) * COUNT(*), 0) AS theta
              FROM cuped_values_2
              GROUP BY 1, 2
            )

            , cuped_metrics_1 as (
              SELECT
                device_id,
                a.metric_name,
                metric_result,
                metric_covariate,
                COALESCE(metric_result - (metric_covariate - covariate_mean) * theta, metric_result) AS cuped_result
              FROM cuped_data_1 AS a
                INNER JOIN cuped_values_1 b
                  ON a.metric_name = b.metric_name
            )

            , cuped_metrics_2 as (
              SELECT
                device_id,
                a.metric_name,
                a.platform_type AS platform,
                metric_result,
                metric_covariate,
                COALESCE(metric_result - (metric_covariate - covariate_mean) * theta, metric_result) AS cuped_result
              FROM cuped_data_2 AS a
                INNER JOIN cuped_values_1 AS b
                  ON a.platform_type = b.platform_type
                  AND a.metric_name = b.metric_name
            )

            , cuped_metrics_3 as (
              SELECT
                device_id,
                a.metric_name,
                a.platform,
                metric_result,
                metric_covariate,
                COALESCE(metric_result - (metric_covariate - covariate_mean) * theta, metric_result) AS cuped_result
              FROM cuped_data_3 AS a
                INNER JOIN cuped_values_2 AS b
                  ON a.platform = b.platform
                  AND a.metric_name = b.metric_name
            )



            , cuped_results AS (
                SELECT metric_name,
                        'ALL' as platform,
                        count(distinct device_id) size,
                        avg(metric_result) result_avg,
                        STDDEV(metric_result) result_std,
                        avg(cuped_result) avg_cuped_result,
                        STDDEV(cuped_result) std_cuped_result
                FROM cuped_metrics_1
                GROUP BY  1, 2

                UNION ALL

                SELECT metric_name,
                       platform,
                       COUNT(distinct device_id) size,
                       AVG(metric_result) result_avg,
                       STDDEV(metric_result) result_std,
                       AVG(cuped_result) avg_cuped_result,
                       STDDEV(cuped_result) std_cuped_result
                FROM cuped_metrics_2
                GROUP BY 1, 2

                UNION ALL

                SELECT metric_name,
                       platform,
                       COUNT(distinct device_id) size,
                       AVG(metric_result) result_avg,
                       STDDEV(metric_result) result_std,
                       AVG(cuped_result) avg_cuped_result,
                       STDDEV(cuped_result) std_cuped_result
                FROM cuped_metrics_3
                GROUP BY 1, 2
            )

            SELECT 
                   metric_name,
                   platform,
                   size AS observations,
                   avg_cuped_result,
                   std_cuped_result
            FROM cuped_results        
            """

# Glue all CTEs together and query

In [12]:
print('choose attribute filter')
attribute_filter_str = interactive(filter_generator().make_sql_where_string, 
                                   field = filter_generator().filter_attributes_choices(), 
                                   condition = filter_generator().attribute_conditions_choices(), 
                                   value = '')
display(attribute_filter_str)

print('')
print('choose metric filter')
metric_filter_str = interactive(filter_generator().make_sql_where_string, 
                                field = filter_generator().filter_metrics_choices(), 
                                condition = filter_generator().metric_conditions_choices(), 
                                value = '')
display(metric_filter_str)

print('')
print('choose your primary metric')
metric_str = interactive(metric_switcher().choose_metric, metric = metric_switcher().possible_metrics())
display(metric_str)


choose attribute filter


interactive(children=(Dropdown(description='field', options=('no filters', 'user_id', 'device_id', 'platform',…


choose metric filter


interactive(children=(Dropdown(description='field', options=('no filters', 'tvt_sec', 'movie_non_autoplay_tvt_…


choose your primary metric


interactive(children=(Dropdown(description='metric', options=('all_tvt_hours', 'capped_tvt', 'new_viewer_first…

In [ ]:
step1 = filter_generator().generate_filter_cte(attribute_sql = attribute_filter_str, metric_sql = metric_filter_str)
step2 = raw_user_data().generate_raw_user_data_cte()
step3 = metric_switcher().generate_user_data_cte(metric_str.result) 
step4 = metric_summary_cte().generate_metric_summary_cte() 
step5 = cuped_ctes().generate_cuped_ctes()

FINAL_SQL = step1 + step2 + step3 + step4 + step5

In [ ]:
print(FINAL_SQL)

In [ ]:
df = tdr.query_redshift(FINAL_SQL).to_df()

Exception ignored in: Exception ignored in: Exception ignored in: Exception ignored in: Exception ignored in: 

In [ ]:
df

# Sample size calculation

In [16]:
def effect(x=0.01):
    return x

def variations(x=1):
    return x

def allocation(x=0.50):
    return x

def power(x=0.8):
    return x

def alpha(x=0.05):
    return x

print('relative effect size')
EFFECT_SIZE_RELATIVE = interactive(effect, x=(0.0,1.0,0.01))
display(EFFECT_SIZE_RELATIVE)

print('')
print('number of treatments')
NUMBER_VARIATIONS = interactive(variations, x=(0,8,1))
display(NUMBER_VARIATIONS)

print('')
print('allocation per variation (including control)')
ALLOCATION = interactive(allocation, x=(0.0,1.0,0.01))
display(ALLOCATION)

print('')
print('power')
POWER = interactive(power, x=(0.0,1.0,0.01))
display(POWER)

print('')
print('alpha')
ALPHA = interactive(alpha, x=(0.0,1.0,0.01))
display(ALPHA)

relative effect size


interactive(children=(FloatSlider(value=0.01, description='x', max=1.0, step=0.01), Output()), _dom_classes=('…


number of treatments


interactive(children=(IntSlider(value=1, description='x', max=8), Output()), _dom_classes=('widget-interact',)…


allocation per variation (including control)


interactive(children=(FloatSlider(value=0.5, description='x', max=1.0, step=0.01), Output()), _dom_classes=('w…


power


interactive(children=(FloatSlider(value=0.8, description='x', max=1.0, step=0.01), Output()), _dom_classes=('w…


alpha


interactive(children=(FloatSlider(value=0.05, description='x', max=1.0, step=0.01), Output()), _dom_classes=('…

In [17]:
# ---------- Constants ---------- #
EFFECT_SIZE_RELATIVE = EFFECT_SIZE_RELATIVE.result
NUMBER_VARIATIONS = NUMBER_VARIATIONS.result
ALLOCATION = ALLOCATION.result
COL_NAME_P = 'avg_cuped_result'
STD_COL_NAME = 'std_cuped_result'
POWER = POWER.result
ALPHA = ALPHA.result
RATIO = 1
SAMPLING = 1  # TODO: make this dynamic between 1 and 1000 for sampled analytics

CORRECTED_ALPHA = ALPHA / NUMBER_VARIATIONS
P2_MULTIPLICATIVE_FACTOR =  1 + EFFECT_SIZE_RELATIVE

In [22]:
# ---------- Functions ---------- #
def sample_power_ttest(p1, p2, sd_diff, alpha=0.05, power=0.8, ratio=1):
    # Constants
    ALTERNATIVE = 'two-sided'

    mean_diff = abs(p2 - p1)
    std_effect_size = mean_diff / sd_diff
    n = tt_ind_solve_power(effect_size=std_effect_size, 
                         alpha=alpha, 
                         power=power, 
                         ratio=ratio, 
                         alternative=ALTERNATIVE)
    return np.array(n).round()    
#     return int(round(n))

    
# ---------- Implementation ---------- #
df['sample_required'] =  df.apply(lambda row: sample_power_ttest(
    p1 = row[COL_NAME_P],
    p2 = row[COL_NAME_P] * P2_MULTIPLICATIVE_FACTOR,
    sd_diff = row[STD_COL_NAME],
    alpha = CORRECTED_ALPHA,
    power = POWER,
    ratio = RATIO)
                                  , axis=1)

df['weeks_required'] = df['sample_required'] / (df['observations'] * 0.5 * ALLOCATION * SAMPLING)

df['avg_cuped_result'] = df['avg_cuped_result'].round(3)
df['std_cuped_result'] = df['std_cuped_result'].round(3)
df['weeks_required'] = df['weeks_required'].round(1)

display(df)

,metric_name,platform,observations,avg_cuped_result,std_cuped_result,sample_required,weeks_required
0,all_tvt_hours,ALL,23619554,5.130,12.192,886759.0,0.2
1,all_tvt_hours,IPHONE,2254961,1.382,4.184,1438343.0,2.6
2,all_tvt_hours,SONY,98288,8.267,17.358,692157.0,28.2
3,all_tvt_hours,OTT,12980460,7.706,14.808,579627.0,0.2
4,all_tvt_hours,XBOXONE,268144,10.080,17.902,495122.0,7.4
5,all_tvt_hours,ROKU,4795116,8.094,13.639,445683.0,0.4
6,all_tvt_hours,COMCAST,1552611,6.165,9.957,409523.0,1.1
7,all_tvt_hours,COX,165752,6.973,10.639,365449.0,8.8
8,all_tvt_hours,VIZIO,746548,3.394,6.797,629327.0,3.4
9,all_tvt_hours,MOBILE,7295231,2.332,6.465,1206511.0,0.7


# APPENDIX

Future improvements: 

In [ ]:
# TODO: make the list of available metrics consistent with exp dash
# Primary experimentation dash metrics:
primary = [
    'activations',
    'ad_impressions',
    'capped_linear_and_vod_tvt_hours_4_hr_per_day',
    'capped_tvt_hours_4_hr_per_day',
    'linear_and_vod_all_user_retained_in_experiment_timeframe',
    'linear_and_vod_viewer_conversion',
    'new_user_1_to_8_days_retained',
    'new_user_capped_first_day_tvt_hours_4_hr_per_day', 
    'new_viewer_first_day_conversion',
    'signups',
    'viewer_5_min_conversion',
    'visits'
]

In [ ]:
# Example events level query
# ANALYTICS_FILTER_QUERY """
#   SELECT
#     device_id,
#     device_first_seen_ts,
#     DATE_TRUNC('day', ts) AS ds,
#     DATEADD('week',-2,DATE_TRUNC('week',GETDATE())) as first_exposure_ds,
#     platform,
#     platform_type,
#     max(case when is_confirmed = 't' then 1 else 0 end) AS metric_value
#   from
#     tubidw.sampled_analytics_thousandth -- sampled table
#   where
#     DATE_TRUNC('week',ts) >= dateadd('week',-4,DATE_TRUNC('week',GETDATE()))
#     and DATE_TRUNC('week',ts) < DATE_TRUNC('week',GETDATE())
    
#     and user_id != device_id -- signed in condition
#     AND UPPER(platform) IN ('WEB', 'IPHONE')
#     AND device_first_seen_ts IS NOT NULL
#   group by
#     1,2,3,4,5,6
# """